# Facebook Graph Processing for SBM Graph Configuration

This notebook describes the process of analyzing a real-world graph to inform the SBM graph configuration. The real-world graph we use is the [Facebook Large Page-Page Network](https://snap.stanford.edu/data/facebook-large-page-page-network.html) from the [Stanford Network Analysis Project](https://snap.stanford.edu/index.html). Extract `musae_facebook_edges.csv` from the downloaded zip, remove the first line, and place it in `data/edges.csv`.

Start by importing `networkx`.

In [1]:
import networkx as nx

Read in the facebook graph.

In [2]:
fb_edges_csv = open("../data/edges.csv", "rb") # read graph
fb_graph = nx.read_edgelist(fb_edges_csv, delimiter=",", nodetype=int)

Determine communities.

In [3]:
from networkx.algorithms.community import greedy_modularity_communities
communities = list(greedy_modularity_communities(fb_graph))

Save communities in `pickle` format.

In [4]:
import pickle
from pathlib import Path
Path('../pickle/fb').mkdir(exist_ok=True)
with open('../pickle/fb/graph.pkl', 'wb') as pickle_file:
    pickle.dump([fb_graph], pickle_file)
with open('../pickle/fb/communities.pkl', 'wb') as pickle_file:
    pickle.dump(communities, pickle_file)

Calculate edge density within communities and across communities.

In [5]:
inside_edges = 0
inside_total = 0
outside_edges = 0
outside_total = 0

for c1 in communities:
    for i in c1:
        for j in c1:
            if i != j and fb_graph.has_edge(i, j):
                inside_edges += 1
                inside_total += 1
            elif i != j:
                inside_total += 1
    for c2 in communities:
        if c1 == c2:
            continue
        for i in c1:
            for j in c2:
                if i != j and fb_graph.has_edge(i,j):
                    outside_edges += 1
                    outside_total += 1
                elif i != j:
                    outside_total += 1

Count number of edges.

In [6]:
print(inside_edges)
print(inside_total)
print(outside_edges)
print(outside_total)

318238
70039570
23408
434838860


Assuming 130 blocks of size 172, determine edge densities needed to match number of within-community and cross-community edges.

In [7]:
print(318238 / (172 * 171 * 130))
print(23408 / (172 * 172 * 130 * 129))

0.0832308110765883
4.7181783381111744e-05
